# StudentCourseDB

## Import the libraries

In [5]:
import sqlite3
import random
import string
import datetime
!pip install faker
from faker import Faker

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 22.0 MB/s eta 0:00:00


## Create DB

In [9]:
DB_NAME = "college.db"
fake = Faker()

## Connect to data base

In [12]:
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

In [14]:
# Enable foreign keys
cursor.execute("PRAGMA foreign_keys = ON;")

## Create Tables

In [17]:
# Table-1 (Students)

cursor.execute("""
CREATE TABLE IF NOT EXISTS students (
    student_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    gender TEXT CHECK(gender IN ('Male','Female','Other')),
    
    -- ORDINAL (ordered categories)
    class_level TEXT CHECK(class_level IN ('Freshman','Sophomore','Junior','Senior')),

    -- INTERVAL (meaningless zero)
    gpa REAL CHECK(gpa >= 0 AND gpa <= 4.0),

    -- RATIO (meaningful zero)
    age INTEGER CHECK(age >= 0),

    major TEXT
);
""")

In [19]:
# Table-2 (Courses)

cursor.execute("""
CREATE TABLE IF NOT EXISTS courses (
    course_id INTEGER PRIMARY KEY AUTOINCREMENT,
    course_name TEXT NOT NULL,
    department TEXT NOT NULL,
    -- RATIO
    course_credits INTEGER CHECK(course_credits > 0)
);
""")

In [25]:
# Table-3 

cursor.execute("""
CREATE TABLE IF NOT EXISTS enrollments (
    student_id INTEGER,
    course_id INTEGER,
    semester TEXT,
    year INTEGER CHECK(year >= 2000),
    grade TEXT CHECK(grade IN ('A','B','C','D','F','W')),

    -- Composite Key
    PRIMARY KEY (student_id, course_id, semester, year),

    FOREIGN KEY (student_id) REFERENCES students(student_id),
    FOREIGN KEY (course_id) REFERENCES courses(course_id)
);
""")

In [27]:
conn.commit()

## Populate course table.

In [30]:
course_list = [
    ("Intro to Psychology", "Psychology", 3),
    ("Calculus I", "Mathematics", 4),
    ("Calculus II", "Mathematics", 4),
    ("Biology I", "Biology", 4),
    ("Organic Chemistry", "Chemistry", 4),
    ("Microeconomics", "Economics", 3),
    ("Macroeconomics", "Economics", 3),
    ("English Composition", "English", 3),
    ("World History", "History", 3),
    ("Computer Science I", "Computer Science", 4),
    ("Data Structures", "Computer Science", 4),
    ("Algorithms", "Computer Science", 4),
    ("Physics I", "Physics", 4),
    ("Physics II", "Physics", 4),
    ("Statistics", "Mathematics", 3),
    ("Art Appreciation", "Arts", 3),
    ("Music Theory", "Music", 3),
    ("Sociology 101", "Sociology", 3),
    ("Marketing Principles", "Business", 3),
    ("Finance 101", "Business", 3)
]

In [32]:
cursor.executemany("""
    INSERT INTO courses (course_name, department, course_credits)
    VALUES (?, ?, ?)
""", course_list)

conn.commit()

## Populate students table with 1000+ rows

In [35]:
class_levels = ["Freshman", "Sophomore", "Junior", "Senior"]
majors = ["Computer Science", "Biology", "Business", "Psychology",
          "Mathematics", "Chemistry", "History", "English"]

genders = ["Male", "Female", "Other"]

students_data = []

for _ in range(1000):
    students_data.append((
        fake.first_name(),
        fake.last_name(),
        random.choice(genders),
        random.choice(class_levels),                
        round(random.uniform(0, 4), 2),             
        random.randint(17, 30),                     
        random.choice(majors)
    ))


In [37]:
cursor.executemany("""
INSERT INTO students (first_name, last_name, gender, class_level, gpa, age, major)
VALUES (?, ?, ?, ?, ?, ?, ?)
""", students_data)

In [39]:
conn.commit()

## Populate enrollments table.

In [42]:
semesters = ["Fall", "Spring", "Summer"]
grades = ["A","B","C","D","F","W"]

enrollments_data = []
all_student_ids = [row[0] for row in cursor.execute("SELECT student_id FROM students").fetchall()]
all_course_ids = [row[0] for row in cursor.execute("SELECT course_id FROM courses").fetchall()]

for student_id in all_student_ids:
    for _ in range(random.randint(2, 4)):
        enrollments_data.append((
            student_id,
            random.choice(all_course_ids),
            random.choice(semesters),
            random.randint(2018, 2025),
            random.choice(grades)
        ))

cursor.executemany("""
INSERT OR IGNORE INTO enrollments (student_id, course_id, semester, year, grade)
VALUES (?, ?, ?, ?, ?)
""", enrollments_data)

In [44]:
conn.commit()

## Close DB

In [47]:
conn.close()

print("Database created successfully:", DB_NAME)

Database created successfully: college.db
